In [25]:
import torch
import torchaudio
from transformers import MimiModel, AutoFeatureExtractor
from torchview import draw_graph

In [26]:
# load the model + feature extractor (for pre-processing the audio)
model = MimiModel.from_pretrained("kyutai/mimi")
feature_extractor = AutoFeatureExtractor.from_pretrained("kyutai/mimi")

# Define the path to your local wav file
# I'll use a file from your project as an example
audio_path = "./music.mp3"

# Load the audio file
waveform_orig, original_sampling_rate = torchaudio.load(audio_path)
num_samples = waveform_orig.shape[1]
print("sample count: ", num_samples)
print(f"Audio sampled at {original_sampling_rate}S/s. lenght: {num_samples / original_sampling_rate}s")

sample count:  21046753
Audio sampled at 48000S/s. lenght: 438.4740208333333s


In [27]:
waveform_orig = waveform_orig[0, :].unsqueeze(0)
waveform_orig.shape

torch.Size([1, 21046753])

In [50]:
sample_seconds = 5
skip_seconds = 15
skip_samples = skip_seconds * original_sampling_rate
waveform = waveform_orig[:, skip_samples:skip_samples+sample_seconds * original_sampling_rate]
waveform.shape

torch.Size([1, 240000])

In [51]:
# Resample the audio to the model's required sampling rate
resampler = torchaudio.transforms.Resample(orig_freq=original_sampling_rate, new_freq=feature_extractor.sampling_rate)
resampled_waveform = resampler(waveform)

# The model expects a 1D array, so we squeeze the tensor
audio_sample = resampled_waveform.squeeze().numpy()
audio_sample.shape

# pre-process the inputs
inputs = feature_extractor(raw_audio=audio_sample, sampling_rate=feature_extractor.sampling_rate, return_tensors="pt")


In [52]:
model_graph = draw_graph(model, input_data=inputs).visual_graph
# Line below triggers an error: "RuntimeError: Failed to run torchgraph"
# model_graph.render(directory='/').replace('\\', '/')




In [23]:
# model_graph

In [55]:

# explicitly encode then decode the audio inputs
encoder_outputs = model.encode(inputs["input_values"], num_quantizers=32)
print("encoder out:", encoder_outputs.audio_codes.shape)
print("encoder out:", encoder_outputs.audio_codes)
audio_values_decoded = model.decode(encoder_outputs.audio_codes)[0]

# or the equivalent with a forward pass
audio_values_forward = model(inputs["input_values"]).audio_values

# You can now save, play, or further process the audio_values
# For example, to save the output as a new wav file:
# torchaudio.save("output_audio.wav", audio_values_forward.cpu(), feature_extractor.sampling_rate)

print("Successfully processed the local audio file.")
print("Shape of the output audio from decoded:", audio_values_decoded.shape)
print("Shape of the output audio from forward pass:", audio_values_forward.shape)

encoder out: torch.Size([1, 32, 63])
encoder out: tensor([[[1049,  605,  319,  ..., 1088,  372,  555],
         [1073, 1577, 1926,  ...,  396,  232,  237],
         [ 834,  834, 1810,  ..., 1748,   38, 1662],
         ...,
         [ 634,  846, 1602,  ...,  576, 1311,  911],
         [ 873,  313, 1173,  ...,  790, 1101, 1636],
         [1902, 1804,  107,  ...,  723,  181, 1924]]])
Successfully processed the local audio file.
Shape of the output audio from decoded: torch.Size([1, 1, 120960])
Shape of the output audio from forward pass: torch.Size([1, 1, 120000])


In [39]:
from IPython.display import display, Audio

In [57]:
display(Audio(data=audio_sample, rate = feature_extractor.sampling_rate, autoplay=False))

In [56]:
display(Audio(data=audio_values_decoded.detach().numpy().squeeze(0), rate = feature_extractor.sampling_rate, autoplay=False))

## MIMI torch compatible

In [1]:
from huggingface_hub import hf_hub_download
import torch

from moshi.models import loaders, LMGen

ModuleNotFoundError: No module named 'moshi'

In [ ]:

mimi_weight = hf_hub_download(loaders.DEFAULT_REPO, loaders.MIMI_NAME)
mimi = loaders.get_mimi(mimi_weight, device='cpu')
mimi.set_num_codebooks(8)  # up to 32 for mimi, but limited to 8 for moshi.

tokenizer-e351c8d8-checkpoint125.safeten(…):   0%|          | 0.00/385M [00:00<?, ?B/s]

In [ ]:
wav = audio_sample

## WARNING: When streaming, make sure to always feed a total amount of audio that is a multiple
#           of the frame size (1920). You should pad or buffer accordingly. Since version 0.2.5a, 
#           Mimi no longer supports partial frames in streaming mode. Besides, when executing on GPU,
#           you should always pass the same amount of audio, as the calls are CUDAGraphed for efficiency.

with torch.no_grad():
    codes = mimi.encode(wav)  # [B, K = 8, T]
    decoded = mimi.decode(codes)

    # Supports streaming too.
    frame_size = mimi.frame_size
    all_codes = []
    with mimi.streaming(batch_size=1):
        for offset in range(0, wav.shape[-1], frame_size):
            frame = wav[:, :, offset: offset + frame_size]
            codes = mimi.encode(frame)
            assert codes.shape[-1] == 1, codes.shape
            all_codes.append(codes)

NameError: name 'audio_sample' is not defined

In [ ]:

# Now if you have a GPU around.
mimi.cuda()
moshi_weight = hf_hub_download(loaders.DEFAULT_REPO, loaders.MOSHI_NAME)
moshi = loaders.get_moshi_lm(moshi_weight, device='cuda')
lm_gen = LMGen(moshi, temp=0.8, temp_text=0.7)  # this handles sampling params etc.
out_wav_chunks = []
# Now we will stream over both Moshi I/O, and decode on the fly with Mimi.
with torch.no_grad(), lm_gen.streaming(1), mimi.streaming(1):
    for idx, code in enumerate(all_codes):
        tokens_out = lm_gen.step(code.cuda())
        # tokens_out is [B, 1 + 8, 1], with tokens_out[:, 1] representing the text token.
        if tokens_out is not None:
            wav_chunk = mimi.decode(tokens_out[:, 1:])
            out_wav_chunks.append(wav_chunk)
        print(idx, end='\r')
out_wav = torch.cat(out_wav_chunks, dim=-1)
